# Water models
Here we will demonstrate how MoSDeF can be used to compare two different force fields; in this case, two atomistic water models

In [ ]:
import mbuild as mb
from foyer import Forcefield

In [ ]:
water = mb.load('O', smiles=True)
water.name = 'water'

In [ ]:
system = mb.fill_box(compound=water, n_compounds=1000, density=993)

In [ ]:
system.to_parmed(residues=['water'])

In [ ]:
SPCE = Forcefield('spce.xml')
TIP3P = Forcefield('tip3p.xml')

In [ ]:
system_spce = SPCE.apply(system.to_parmed(residues=['water']))
system_tip3p = TIP3P.apply(system.to_parmed(residues=['water']))

In [ ]:
system_spce.save('water_spce.gro')
system_spce.save('water_spce.top')
system_tip3p.save('water_tip3p.gro')
system_tip3p.save('water_tip3p.top')

In [1]:
!gmx grompp -c water_spce.gro -p water_spce.top -f npt.mdp -o water_spce.tpr && \
 gmx grompp -c water_tip3p.gro -p water_tip3p.top -f npt.mdp -o water_tip3p.tpr

    :-) GROMACS - gmx grompp, 2020-beta1-dev-20191030-7aad04af46-unknown (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the proj

In [ ]:
!gmx mdrun -v -deffnm water_spce && \
 gmx mdrun -v -deffnm water_tip3p

In [ ]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid.inset_locator import InsetPosition

In [ ]:
trj_spce = md.load('water_spce.xtc', top='water_spce.gro')
trj_tip3p = md.load('water_tip3p.xtc', top='water_tip3p.gro')

In [ ]:
!echo 4 | gmx energy -f water_spce.edr -o energy_spce.xvg

In [ ]:
!echo 4 | gmx energy -f water_tip3p.edr -o energy_tip3p.xvg

In [ ]:
fig, ax = plt.subplots()

for model in ['spce', 'tip3p']:
    data = np.loadtxt(f'energy_{model}.xvg', comments=['#', '@'])

    ax.plot(data[:, 0], data[:, 1])
    axins = plt.axes([0,0,1,1])

    ip = InsetPosition(ax, [0.2,0.2,0.7,0.7])
    axins.set_axes_locator(ip)

    axins.plot(data[100:, 0], data[100:, 1])

In [ ]:
fig, ax = plt.subplots()

for model in ['spce', 'tip3p']:
    trj = md.load(
        f'water_{model}.xtc',
        top=f'water_{model}.gro',
    )

    rho = md.density(trj)

    ax.plot(trj.time, rho)
    axins = plt.axes([0,0,1,1])

    ip = InsetPosition(ax, [0.2,0.2,0.7,0.7])
    axins.set_axes_locator(ip)

    axins.plot(trj.time[100:], rho[100:])

In [ ]:
pairs = trj.top.select_pairs('name O', 'name O')
r, g_r = md.compute_rdf(trj, pairs)

In [ ]:
fig, ax = plt.subplots()

for model in ['spce', 'tip3p']:
    trj = md.load(
        f'water_{model}.xtc',
        top=f'water_{model}.gro',
    )

    pairs = trj.top.select_pairs('name O', 'name O')

    r, g_r = md.compute_rdf(trj[100:], pairs)

    plt.plot(r, g_r)